<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

38 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-03-03 08:36:17--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?Z_sMGRCL7rkF9YCXvLH5QQdHFtRJvGi7bYwzPDctXA4WOUfvrIx0Zai5_GqbY14L_IIWPRrcegMU-Av3yNx8eJvxLDUAVW73iYclWKisEaHeiavWvUAxWAIU-Vk-VeCaq17F8pvEGPNOzQqgAt8eGm9W5Agp5PgYfpxS4_9yv2MqoMfFw5lER5xw241z-_kOWNULVDSiFiYTYv_zCpJLNuvD2w [following]
--2019-03-03 08:36:18--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?Z_sMGRCL7rkF9YCXvLH5QQ

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wb0wv4df
  Stored in directory: /tmp/pip-ephem-wheel-cache-owuljrw1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

In [63]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 2000
#define N 64

#define e 2.7182818

#define blockZSize 4

#define blockXSize 32
#define blockYSize 32

#define tileXSize 16
#define tileYSize 16

#define extN (N + 2 * blockZSize)
#define Dx 1
#define h ((float) Dx / N)
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + blockZSize;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + blockZSize;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  float* helper = new float[N1 * N2];
  
  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
			helper[i * N1 + j] = matrix[extN * (iBegin - blockZSize + i) + jBegin - blockZSize + j];
		}
	}
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
		  int offset = i % 2;
		  if (isBlack) {
			  offset = 1 - offset;
		  }
		  for (int j = k + offset; j < N2 - k; j += 2) {
			  helper[N1 * i + j] = 0.25 * (
            helper[N1 * (i + 1) + j]
          + helper[N1 * (i - 1) + j]
          + helper[N1 * i + j + 1]
				  + helper[N1 * i + j - 1] 
          - h2 * f(iBegin - blockZSize + i, jBegin - blockZSize + j)
        );
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);

	for (int i = iBegin; i < iBegin + tileXSize; i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIIndex = i - iBegin + blockZSize;
      int helperJIndex = j - jBegin + blockZSize; 
			matrix[extN * i + j] = 0.25 * (
          helper[N1 * (helperIIndex + 1) + helperJIndex]
        + helper[N1 * (helperIIndex - 1) + helperJIndex]
        + helper[N1 * helperIIndex + helperJIndex + 1]
				+ helper[N1 * helperIIndex + helperJIndex - 1] 
        - h2 * f(i - blockZSize + 1, j - blockZSize + 1)
      );
		}
	}
    
  delete [] helper;  
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int i = k; i < row - k; i++) {
      matrix[k * row + i] = u_t((i - blockZSize + 1) * h);
      matrix[(row - 1 - k) * row + i] = u_d((i - blockZSize + 1) * h);
    }
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int j = k; j < col - k; j++) {
      matrix[j * col + k] = u_l((j - blockZSize + 1) * h);
      matrix[j * col + row - 1 - k] = u_r((j - blockZSize + 1) * h);
	  }
  }
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;

  for (int i = blockZSize; i < extN - blockZSize; i++) {
		for (int j = blockZSize; j < extN - blockZSize; j++) {
      float fCurrent = func(i - blockZSize + 1, j - blockZSize + 1);
		  tolerance += (matrix1[i * extN + j] - fCurrent) * (matrix1[i * extN + j] - fCurrent);
		}
	}

	return tolerance;
}

int main() {
  ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
  

  for (size_t i = 0; i < extN; i++) {
		for (size_t j = 0; j < extN; j++) {
      myfile << matrixHost[i * extN + j] << " ";
		}
    myfile << "\n";
	}



	float* matrixDevice;
	
	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);

		cudaPeekAtLastError();
		cudaDeviceSynchronize();

		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

      
    cout << calculateDifference(matrixHost) << "\n";
   
    for (size_t i = 0; i < extN; i++) {
		  for (size_t j = 0; j < extN; j++) {
        myfile << matrixHost[i * extN + j] << " ";
		  }
      myfile << "\n";
	 }
   
     
	}
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
  myfile.close();
	return 0;
}


'39963.4\n38037.9\n36558.7\n35329.5\n34265.5\n33320.9\n32467.5\n31686.8\n30965.6\n30294.4\n29665.9\n29074.2\n28514.8\n27984.1\n27478.9\n26996.8\n26535.4\n26093\n25667.9\n25258.8\n24864.4\n24483.7\n24115.5\n23759.4\n23414.2\n23079.4\n22754.4\n22438.6\n22131.4\n21832.5\n21541.3\n21257.5\n20980.7\n20710.6\n20446.8\n20189.2\n19937.3\n19691\n19449.9\n19214.1\n18983\n18756.7\n18535\n18317.6\n18104.3\n17895.1\n17689.9\n17488.3\n17290.4\n17096\n16905.1\n16717.4\n16532.9\n16351.5\n16173.1\n15997.6\n15824.9\n15655\n15487.8\n15323.1\n15161\n15001.3\n14843.9\n14689\n14536.3\n14385.7\n14237.4\n14091.1\n13946.9\n13804.7\n13664.5\n13526.2\n13389.7\n13255\n13122.2\n12991\n12861.6\n12733.9\n12607.8\n12483.3\n12360.3\n12238.9\n12119\n12000.6\n11883.7\n11768.1\n11653.9\n11541.2\n11429.7\n11319.6\n11210.8\n11103.2\n10996.9\n10891.9\n10788\n10685.3\n10583.8\n10483.5\n10384.2\n10286.1\n10189.1\n10093.1\n9998.21\n9904.38\n9811.55\n9719.72\n9628.92\n9539.05\n9450.19\n9362.27\n9275.27\n9189.22\n9104.07\n9019.8

CODE (AT LEAST NOW)

In [38]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 34 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(14), end='')
  i += 1




             1       1.03174       1.06449       1.09829       1.13315       1.16912       1.20623       1.24452       1.28403       1.32478       1.36684       1.41023       1.45499       1.50118       1.54883         1.598       1.64872       1.70106       1.75505       1.81077       1.86825       1.92755       1.98874       2.05187         2.117        2.1842       2.25353       2.32507       2.39888       2.47502       2.55359       2.71828             
       1.03174             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0       2.80457             
       1.06449             0             0             0             0    